In [21]:
import time

import numpy as np

def rotate_householder(x, y, S):
    """
    Applies the shortest-path rotation mapping x -> y to set S.
    S is an array of shape (n_samples, n_dims).
    """
    # 1. First reflection: x -> -x
    u1 = x / np.linalg.norm(x)
    
    # 2. Second reflection: -x -> y
    v2 = y - (-x)
    u2 = v2 / np.linalg.norm(v2)
    
    # Vectorized application: H(s) = s - 2(u·s)u
    # We apply H1 then H2
    S_temp = S - 2 * np.outer(S @ u1, u1)
    S_rotated = S_temp - 2 * np.outer(S_temp @ u2, u2)
    
    return S_rotated


def rotate_set_qr(x, y, S):
    """
    Maps x -> y using QR Decomposition. 
    Complexity: O(n^3) to find Q, O(n^2) per vector in S.
    """
    n = len(x)
    # Create a basis starting with y
    M = np.eye(n)
    M[:, 0] = y
    
    # QR gives an orthogonal matrix Q where Q[:, 0] is proportional to y
    Q, _ = np.linalg.qr(M)
    
    # QR doesn't guarantee the sign of the first column. Fix it:
    if not np.allclose(Q[:, 0], y):
        Q[:, 0] = -Q[:, 0] # This might flip det to -1

    # Force det(Q) == 1 to ensure pure rotation (not reflection)
    if np.linalg.det(Q) < 0:
        Q[:, -1] = -Q[:, -1]
        
    # Rotate S: S_rotated = S * Q^T
    return S @ Q.T

# --- Validation ---
n = 100
# x = np.array([1.0, 0.0] + [0.0] * (n - 2))
# y = np.array([0.0, 1.0] + [0.0] * (n - 2)) # 90-degree turn in XY plane
x = np.random.randn(n)
x /= np.linalg.norm(x)
y = np.random.randn(n)
y /= np.linalg.norm(y)

# Set S: a point at 30 degrees (pi/6)
S = np.random.randn(1, n)
S /= np.linalg.norm(S, axis=-1, keepdims=True)

start = time.time()
S_hh = rotate_householder(x, y, S)
end = time.time()
print(f"Householder time: {end - start}")

start = time.time()
S_qr = rotate_set_qr(x, y, S)
end = time.time()
print(f"QR time: {end - start}")

# The shapes (distances from the center) should be identical for hh but not necessarily for qr
dist_x = np.linalg.norm(S - x, axis=1)
dist_y_qr = np.linalg.norm(S_qr - y, axis=1)
dist_y_hh = np.linalg.norm(S_hh - y, axis=1)

print(np.allclose(dist_x, dist_y_qr)) # This should necessarily be True
print(np.allclose(dist_x, dist_y_hh)) # This should be True

Householder time: 0.00017595291137695312
QR time: 0.08390355110168457
False
True


In [23]:
# 1. Check dot products with the "center" vector
# Before: dot(S, x) | After: dot(S_rotated, y)
dots_before = S @ x
dots_after_hh = S_hh @ y
dots_after_qr = S_qr @ y

print(f"HH preserves angles to center: {np.allclose(dots_before, dots_after_hh)}")
print(f"QR does not preserve angles to center: {np.allclose(dots_before, dots_after_qr)}")

# 2. Check internal pairwise distances (the 'shape' of the cloud)
from scipy.spatial.distance import pdist

shape_before = pdist(S)
shape_after_hh = pdist(S_hh)

print(f"HH preserves internal shape:   {np.allclose(shape_before, shape_after_hh)}")

HH preserves angles to center: True
QR does not preserve angles to center: False
HH preserves internal shape:   True


In [13]:
import numpy as np

def rotate_householder(x, y, S):
    """
    Applies the shortest-path rotation mapping x -> y to set S.
    S is an array of shape (n_samples, n_dims).
    """
    # 1. First reflection: x -> -x
    u1 = x / np.linalg.norm(x)
    
    # 2. Second reflection: -x -> y
    v2 = y - (-x)
    u2 = v2 / np.linalg.norm(v2)
    
    # Vectorized application: H(s) = s - 2(u·s)u
    # We apply H1 then H2
    S_temp = S - 2 * np.outer(S @ u1, u1)
    S_rotated = S_temp - 2 * np.outer(S_temp @ u2, u2)
    
    return S_rotated

# --- Setup ---
n_dims = 10
n_samples = 1000
sigma = 0.1  # Spread of the cluster

# Define x and a random unit vector y
x = np.zeros(n_dims)
x[0] = 1.0

y = np.random.randn(n_dims)
y /= np.linalg.norm(y)

# 1. Create set S centered at x
# Start with x, add noise to all dims, then normalize
noise = np.random.normal(0, sigma, (n_samples, n_dims))
S = (x + noise) 
S /= np.linalg.norm(S, axis=1)[:, np.newaxis]

# 2. Rotate the set
S_rotated = rotate_householder(x, y, S)

# 3. Validation
mean_direction_before = np.mean(S, axis=0)
mean_direction_before /= np.linalg.norm(mean_direction_before)

mean_direction_after = np.mean(S_rotated, axis=0)
mean_direction_after /= np.linalg.norm(mean_direction_after)

print(f"Target vector y:            {y[:5]}...") # Showing first 5 dims
print(f"Mean direction after rot:   {mean_direction_after[:5]}...")
print(f"Cosine Similarity (should be ~1.0): {np.dot(mean_direction_after, y):.6f}")
print()
# show values for before
print(f"Source vector x:            {x[:5]}...")
print(f"Mean direction before rot:   {mean_direction_before[:5]}...")
print(f"Cosine Similarity (should be ~1.0): {np.dot(mean_direction_before, x):.6f}")

Target vector y:            [-0.2944008  -0.21862317 -0.22352029 -0.16052253  0.1493813 ]...
Mean direction after rot:   [-0.28425431 -0.21634951 -0.22374479 -0.16494304  0.15293105]...
Cosine Similarity (should be ~1.0): 0.999916

Source vector x:            [1. 0. 0. 0. 0.]...
Mean direction before rot:   [ 0.9999158   0.00539136  0.00296304 -0.00213135  0.00141947]...
Cosine Similarity (should be ~1.0): 0.999916
